<a href="https://colab.research.google.com/github/Rewanthnayak/Jupyer_notebooks/blob/main/Pinecone_Database_movie_dialouges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Vector databases** are purpose-built to handle the unique structure of **vector embeddings**. They **index vectors** for **easy search and retrieval** by comparing values and finding those that are most similar to one another.

In [ ]:
!pip install sentence-transformers pinecone-client langchain -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.4 MB/s eta 0:00:0

In [ ]:
import pandas as pd

In [ ]:
import os
import io
import urllib
import zipfile

def load_cornell_movie_dialogs():
    """Loads the Cornell Movie Dialogue Corpus into a Pandas DataFrame."""

    url = "http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip"
    file_path = "cornell_movie_dialogs.zip"

    if not os.path.exists(file_path):
        print("Downloading Cornell Movie Dialogue Corpus...")
        urllib.request.urlretrieve(url, file_path)

    with zipfile.ZipFile(file_path) as zip_file:
        with zip_file.open('cornell movie-dialogs corpus/movie_lines.txt') as lines_file:
            lines = lines_file.readlines()
            lines = [line.decode("unicode_escape") for line in lines]

        with zip_file.open('cornell movie-dialogs corpus/movie_conversations.txt') as conversations_file:
            conversations = conversations_file.readlines()
            conversations = [conv.decode("unicode_escape") for conv in conversations]

    # return conversations
    df = pd.DataFrame({
        "LineID": [line.split(" +++$+++ ")[0] for line in lines],
        "Character": [line.split(" +++$+++ ")[1] for line in lines],
        "Movie": [line.split(" +++$+++ ")[2] for line in lines],
        "Name": [line.split(" +++$+++ ")[3] for line in lines],
        "Line": [line.split(" +++$+++ ")[4] for line in lines],
    })

    # df["ConversationID"] = [conversation.split(" +++$+++ ")[0] for conversation in conversations]
    # df["UtteranceID"] = [conversation.split(" +++$+++ ")[1] for conversation in conversations]

    return df

df = load_cornell_movie_dialogs()
df

,LineID,Character,Movie,Name,Line
0,L1045,u0,m0,BIANCA,They do not!\n
1,L1044,u2,m0,CAMERON,They do to!\n
2,L985,u0,m0,BIANCA,I hope so.\n
3,L984,u2,m0,CAMERON,She okay?\n
4,L925,u0,m0,BIANCA,Let's go.\n
...,...,...,...,...,...
304708,L666371,u9030,m616,DURNFORD,Lord Chelmsford seems to want me to stay back ...
304709,L666370,u9034,m616,VEREKER,I'm to take the Sikali with the main column to...
304710,L666369,u9030,m616,DURNFORD,"Your orders, Mr Vereker?\n"
304711,L666257,u9030,m616,DURNFORD,"Good ones, yes, Mr Vereker. Gentlemen who can ..."


In [ ]:
df.isnull().sum()

LineID       0
Character    0
Movie        0
Name         0
Line         0
dtype: int64

In [ ]:
import pinecone
pinecone.init(api_key='f651e399-7355-4972-9595-afb52787ecf1', environment='asia-southeast1-gcp-free')

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2',device='cpu')

In [ ]:
embeding = model.encode("This is a sentence")
# print(embeding)
print(len(embeding))

embeding = model.encode("This is not a sentence")
# print(embeding)
print(len(embeding))

768
768


In [ ]:
pinecone.create_index(name='movie-dialogues', dimension=768,metric='cosine')

In [ ]:
index = pinecone.Index('movie-dialogues')

In [ ]:
# [(id,vector,metadata)]

In [ ]:
def first_500(input:str):
  if len(input)>512:
    return input[:510]
  else:
    return input

In [ ]:
movie_lines = []
for i,row in data.iterrows():
  if line := row['line']:
    line = first_500(line)
    try :
        movie_lines.append(
                             (str(row['lid']),
                              model.encode(line).tolist(),
                              {
                                  'cid': row['cid'],
                                  'mid' : row['mid'],
                                  'character' : row['character'],
                                  'line': row['line']
                              })
                          )
    except Exception:
          print(f'{Exception} in line : {i}')

    if len(movie_lines)==50 or len(movie_lines)==len(data):
      index.upsert(vectors=movie_lines)
      movie_lines = []

In [ ]:
print(len(movie_lines))

In [ ]:
query = "How do I prepare for civil service?"
xq = model.encode([query]).tolist()
result = index.query(xq, top_k=2, includeMetadata=True)
result

In [ ]:
query = "How to levergae internet for business"
xq = model.encode([query]).tolist()
result = index.query(xq, top_k=2, includeMetadata=True)
result

In [ ]:
!pip install langchain

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
import json

embeddings = HuggingFaceEmbeddings()

In [ ]:
res = embeddings.embed_query('the name is nayak, rewanth nayak.')

In [ ]:
type(res)

list

In [ ]:
res

In [ ]:
query = "i'll have my vengence in this life or the next"
xq = embeddings.embed_query(query)
result = index.query(xq, top_k=8, includeMetadata=True)
result

In [ ]:
request = {
  "topK": 10,
  "includeMetadata": True,
  "namespace": "example_namespace",
  "queries": [
    {
      "filter": {
        "colors": {
          "$eq": "pink"
        }
      },
      "values": res
    }
  ]
}

In [ ]:
request

In [ ]:
json_string = json.dumps(request, indent=4)

print(json_string)

In [ ]:
with open("request.json", "w") as f:
        f.write(json_string)